> Aaron's Experiments with `Closure App - part 2`

### Closure Applications (Part 2)

#### Example 1

Let's write a small function that can increment a counter for us - we don't have an incrementor in Python 

In [2]:
def counter(initial_value):
    # initial value is a local variable here
    
    def inc(increment=1):
        nonlocal initial_value
        # initial_value is a nonlocal (captured) variable here
        initial_value += increment
        return initial_value
    
    return inc

In [4]:
counter1 = counter(0)


In [5]:
print(counter1(0))

0


In [6]:
print(counter1())

1


In [7]:
counter(0)

<function __main__.counter.<locals>.inc(increment=1)>

In [8]:
counter1()

2

In [9]:
counter1(8)

10

In [10]:
counter2 = counter(1000)

In [11]:
counter2(0)

1000

In [12]:
counter2(1)

1001

In [14]:
counter2(220)

1221

In [15]:
counter2(999)

2220

As you can see, each closure maintains a reference to the initial_value that was created when the counter function was called

each time that function was called, a new local variable initial_value was created( with a value assigned form the argument)

and it become a nonlocal (caputred) variable in the inner scope

#### Example 2

Let's modify this example to now build something that can run, and maintain a count of how many times we have run some function.

In [18]:
def counter(fn):
    cnt = 0 # initial fn has been run zero times
    
    def inner(*args, **kwargs):
        nonlocal cnt
        cnt = cnt + 1
        print(f'{fn.__name__} has been called {cnt} times')
        return fn(*args, **kwargs)

    return inner

In [19]:
def add(a, b):
    return a + b

In [22]:
counted_add = counter(add)

We can call the counted _add function:


In [24]:
counted_add(5, 6)

add has been called 1 times


11

And free varialbes are:


In [25]:
counted_add.__code__.co_freevars

('cnt', 'fn')

In [26]:
counted_add(1, 2)

add has been called 2 times


3

In [27]:
counted_add(3, 4)

add has been called 3 times


7

In [28]:
counted_add(100, 100)

add has been called 4 times


200

In [29]:
def mult(a, b, c):
    return a * b * c

In [30]:
counted_mult = counter(mult)

In [31]:
counted_mult(1, 2, 3)

mult has been called 1 times


6

In [32]:
counted_mult.__code__.co_freevars

('cnt', 'fn')

In [34]:
counted_mult(1, 2, 4)

mult has been called 3 times


8

In [35]:
counted_mult(3,3,3)

mult has been called 4 times


27

In [36]:
counted_mult(100, 100, 100)

mult has been called 5 times


1000000

#### Example 3

Let's take this one step further, and actually store the function name and the number of calls in a global dictionary instead of just printing it out all the time.

In [39]:
counters = dict()

def counter(fn):
    cnt = 0  # initally fn has been run zero times

    def inner(*args, **kwargs):
        nonlocal cnt
        cnt += 1
        counters[fn.__name__] = cnt  # counters is global
        return fn(*args, **kwargs)
    
    return inner

In [40]:
counted_add = counter(add)
counted_mult = counter(mult)

Note that counters is a global variable, and therefulre not a free variable:

In [41]:
counted_add.__code__.co_freevars

('cnt', 'fn')

In [42]:
counted_mult.__code__.co_freevars

('cnt', 'fn')

We can now call out functions:

In [43]:
counted_add(1, 2)

3

In [44]:
counted_add(2, 3)

5

In [45]:
counted_mult(1, 2, 'a')

'aa'

In [46]:
counted_mult(2, 3, 'b')

'bbbbbb'

In [47]:
counted_mult(1, 2, 'xyz')

'xyzxyz'

In [48]:
print(counters)

{'add': 2, 'mult': 3}


Of course, this relies on us creating the counters  global variable first and making sure we are naming it that way, so instead, we're going to pass it as an argument to the counter functions:


In [51]:
def counter(fn, counters):
    cnt = 0  # initially fn has been run zero times

    def inner(*args, **kwargs):
        nonlocal cnt
        cnt += 1
        counters[fn.__name__] = cnt  # counters is global
        return fn(*args, **kwargs)
    
    return inner

In [65]:
func_counters = dict()
counted_add = counter(add, func_counters)
counted_mult = counter(mult, func_counters)

In [66]:
counted_add.__code__.co_freevars

('cnt', 'counters', 'fn')

In [67]:
counted_mult.__code__.co_freevars

('cnt', 'counters', 'fn')

As you can see, counters is now a free variable.

We can now call our functions:

In [68]:
for i in range(5):
    counted_add(i, i)

for i in range(10):
    counted_mult(i, i, i)

In [69]:
print(func_counters)

{'add': 5, 'mult': 10}


In [64]:
del func_counters

Of course, we don't have assign the "counted" version of our functions a new name - we can simply assign it to the same name!

In [75]:
def fact(n):
    product = 1
    for i in range(2, n+1):
        product *= i
    return product

In [76]:
fact = counter(fact, func_counters)

In [77]:
fact(0)

1

In [78]:
fact(4)

24

In [79]:
fact(3)

6

In [80]:
print(func_counters)

{'add': 5, 'mult': 10, 'fact': 3}


NOtice, how we essentially added some functionality to our fact function, without modifying what the fact function actually returns.


This leads us straight into our next opic: decorators!


#### Example 1

Let's write a small function that can increase a counter for us - we don't have an incrementor in Python (the ++ operator in Java or C++ for example)

In [1]:
def counter(initial_value):
    # initial value is a local variable here 

    def inc(increment=1):
        nonlocal initial_value
        # initial_value is a nonlocal (captured) variable here
        initial_value += increment
        return initial_value
    
    return inc

In [2]:
counter1 = counter(0)

In [3]:
print(counter1(0))

0


In [4]:
print(counter1())

1


In [5]:
print(counter1())

2


In [7]:
print(counter1(8))

10


In [8]:
counter2 = counter(1000)

In [9]:
print(counter2())

1001


In [10]:
print(counter2())

1002


In [11]:
print(counter2(220))

1222


As you can see, each closure maintains a reference to the inital_value variable that was created when the counter function was called - each time that function was called, a new local variable intial_value was created (with a value assigned from the argument), and it became a nonlocal (captured) variable in the inner scope.

Example 2

Let's modify this example to now build something that can run, and maintain a count of how many times we have run some function.

In [12]:
def counter(fn):
    cnt = 0 # initally fn has been run zero times

    def inner(*args, **kwargs):
        nonlocal cnt
        cnt = cnt + 1
        print(f"{fn.__name__} has been called {cnt} times.")
        return fn(*args, **kwargs)

    return inner

In [13]:
def add(a, b):
    return a + b

And the free variables are:

In [15]:
counted_add = counter(add)

In [18]:
counted_add.__code__.co_freevars

('cnt', 'fn')

We can now call the `counted_add` function:

In [19]:
counted_add(1, 2)

add has been called 1 times.


3

In [20]:
counted_add(2, 3)

add has been called 2 times.


5

In [21]:
def mult(a, b, c):
    return a * b * c

In [22]:
counted_mult = counter(mult)

In [23]:
counted_mult(1, 2, 3)

mult has been called 1 times.


6

In [24]:
counted_mult(2, 3, 4)

mult has been called 2 times.


24

#### Example 3

Let's take this one step further.

We actually `store` the function name and the number of calls `in a global dictionary` instead of just printing it out all the time

In [33]:
counters = dict()

def counter(fn):
    cnt = 0 # initially fn has been run zero times

    def inner(*args, **kwargs):
        nonlocal cnt
        cnt = cnt + 1
        counters[fn.__name__] = cnt # counters is global
        return fn(*args, **kwargs)

    return inner

In [34]:
counted_add = counter(add)
counter_mult = counter(mult)

Note that `counters` is a global variable, and therefore not a free variable:

In [35]:
counted_add.__code__.co_freevars

('cnt', 'fn')

In [36]:
counted_mult.__code__.co_freevars

('cnt', 'fn')

We can now call out functions:

In [38]:
counted_add(1, 2)

3

In [39]:
counted_add(2, 3)

5

In [40]:
counted_mult(1, 2, 'a')


mult has been called 3 times.


'aa'

In [41]:
counted_mult(2, 3, 'b')

mult has been called 4 times.


'bbbbbb'

In [42]:
counted_mult(1, 1, 'abc')

mult has been called 5 times.


'abc'

In [43]:
print(counters)

{'add': 3}


Of course this relies on us creating the counters global variable first and making sure we are naming it that way. So instead, we're going to pass it as an argument to the counter function:

In [54]:
def counter(fn, counters):
    cnt = 0 # initially fn has been run zero time

    def inner(*args, **kwargs):
        nonlocal cnt 
        cnt += 1
        counters[fn.__name__] = cnt
        return fn(*args, **kwargs)
    
    return inner

In [55]:
func_counters = dict()
counted_add = counter(add, func_counters)
counted_mult = counter(mult, func_counters)

In [56]:
counted_add.__code__.co_freevars

('cnt', 'counters', 'fn')

As you can see, `counters` is now a `free variable`

We can now call our functions:

In [57]:
for i in range(5):
    counted_add(i, i)

for i in range(10):
    counted_mult(i, i, i)

In [58]:
print(func_counters)

{'add': 5, 'mult': 10}


Of course, we don't have to assign the `counted` version of our functions a new name - we can simply assign it to the same name!

In [59]:
def fact(n):
    product = 1
    for i in range(2, n+1):
        product *= i
    return product
    

In [60]:
fact = counter(fact, func_counters)

In [61]:
fact(0)

1

In [62]:
fact(3)

6

In [63]:
fact(4)

24

In [64]:
print(func_counters)

{'add': 5, 'mult': 10, 'fact': 3}


Notice, how we essentially added some functionality to our fact function, without modifying what the fact function actually returns.

This leads us straight into our next topic: `decorators`!

### Closure Applications (Part 2)

#### Example 1

Let's write a small function that can increment a counter for us - we don't have an incrementor in Python (the ++ operator in Java or C++ for example):

In [2]:
def counter(initial_value):
    # initial_value is a local variable here
    
    def inc(increment=1):
        nonlocal initial_value
        # initial_value is a nonlocal (captured) variable here
        initial_value += increment
        return initial_value
    
    return inc

In [3]:
counter1 = counter(0)

In [4]:
print(counter1(0))

0


In [5]:
print(counter1())

1


In [6]:
print(counter1())

2


In [7]:
print(counter1(8))

10


In [8]:
counter2 = counter(1000)

In [9]:
print(counter2(0))

1000


In [10]:
print(counter2(1))

1001


In [11]:
print(counter2())

1002


In [12]:
print(counter2(220))

1222


As you can see, each closure maintains a reference to the **initial_value** variable that was created when the **counter** function was **called** - each time that function was called, a new local variable **initial_value** was created (with a value assigned from the argument), and it became a nonlocal (captured) variable in the inner scope.

#### Example 2

Let's modify this example to now build something that can run, and maintain a count of how many times we have run some function.

In [13]:
def counter(fn):
    cnt = 0  # initially fn has been run zero times
    
    def inner(*args, **kwargs):
        nonlocal cnt
        cnt = cnt + 1
        print('{0} has been called {1} times'.format(fn.__name__, cnt))
        return fn(*args, **kwargs)
    
    return inner

In [14]:
def add(a, b):
    return a + b

In [15]:
counted_add = counter(add)

And the free variables are:

In [16]:
counted_add.__code__.co_freevars

('cnt', 'fn')

We can now call the `counted_add` function:

In [17]:
counted_add(1, 2)

add has been called 1 times


3

In [18]:
counted_add(2, 3)

add has been called 2 times


5

In [19]:
def mult(a, b, c):
    return a * b * c

In [20]:
counted_mult = counter(mult)

In [21]:
counted_mult(1, 2, 3)

mult has been called 1 times


6

In [22]:
counted_mult(2, 3, 4)

mult has been called 2 times


24

#### Example 3

Let's take this one step further, and actually store the function name and the number of calls in a global dictionary instead of just printing it out all the time.

In [35]:
counters = dict()

def counter(fn):
    cnt = 0  # initially fn has been run zero times
    
    def inner(*args, **kwargs):
        nonlocal cnt
        cnt = cnt + 1
        counters[fn.__name__] = cnt  # counters is global
        return fn(*args, **kwargs)
    
    return inner

In [26]:
counted_add = counter(add)
counted_mult = counter(mult)

Note that `counters` is a **global** variable, and therefore **not** a free variable:

In [27]:
counted_add.__code__.co_freevars

('cnt', 'fn')

In [28]:
counted_mult.__code__.co_freevars

('cnt', 'fn')

We can now call out functions:

In [29]:
counted_add(1, 2)

3

In [30]:
counted_add(2, 3)

5

In [31]:
counted_mult(1, 2, 'a')

'aa'

In [32]:
counted_mult(2, 3, 'b')

'bbbbbb'

In [33]:
counted_mult(1, 1, 'abc')

'abc'

In [34]:
print(counters)

{'add': 2, 'mult': 3}


Of course this relies on us creating the **counters** global variable first and making sure we are naming it that way, so instead, we're going to pass it as an argument to the **counter** function:

In [36]:
def counter(fn, counters):
    cnt = 0  # initially fn has been run zero times
    
    def inner(*args, **kwargs):
        nonlocal cnt
        cnt = cnt + 1
        counters[fn.__name__] = cnt  # counters is nonlocal
        return fn(*args, **kwargs)
    
    return inner

In [33]:
func_counters = dict()
counted_add = counter(add, func_counters)
counted_mult = counter(mult, func_counters)

In [34]:
counted_add.__code__.co_freevars

('cnt', 'counters', 'fn')

As you can see, `counters` is now a free variable.

We can now call our functions:

In [35]:
for i in range(5):
    counted_add(i, i)

for i in range(10):
    counted_mult(i, i, i)

In [36]:
print(func_counters)

{'add': 5, 'mult': 10}


Of course, we don't have to assign the "counted" version of our functions a new name - we can simply assign it to the same name!

In [37]:
def fact(n):
    product = 1
    for i in range(2, n+1):
        product *= i
    return product

In [38]:
fact = counter(fact, func_counters)

In [39]:
fact(0)

1

In [40]:
fact(3)

6

In [41]:
fact(4)

24

In [42]:
print(func_counters)

{'add': 5, 'mult': 10, 'fact': 3}


Notice, how we essentially **added** some functionality to our `fact` function, without modifying what the `fact` function actually returns.

This leads us straight into our next topic: decorators!